# Setup (on collab)

In [ ]:
!git clone https://github.com/alicebarbosam/fraud-detection-kaggle.git
%cd fraud-detection-kaggle

In [9]:
import os
import joblib
import pandas as pd

model = joblib.load("../models/xgb_optuna_final.joblib")

test_df = pd.read_csv("../data/test.csv")

X_test = test_df.drop(columns=["id"])
test_ids = test_df["id"]

test_probs = model.predict_proba(X_test)[:, 1]

submission = pd.DataFrame({
    "id": test_ids,
    "target": test_probs
})

os.makedirs("../outputs", exist_ok=True)
submission.to_csv("../outputs/submission.csv", index=False)

print("Saved: ../outputs/submission.csv")
print(submission.head())

Saved: ../outputs/submission.csv
       id    target
0  263021  0.002804
1   11379  0.003162
2  147284  0.010335
3  219440  0.002083
4   36940  0.016911


In [10]:
print(submission["target"].describe())
print("max:", submission["target"].max())
print("min:", submission["target"].min())
print("count > 0.5:", (submission["target"] > 0.5).sum())
print("count > 0.1:", (submission["target"] > 0.1).sum())
print("count > 0.01:", (submission["target"] > 0.01).sum())

count    56962.000000
mean         0.007002
std          0.047834
min          0.001878
25%          0.002217
50%          0.002580
75%          0.003548
max          0.997845
Name: target, dtype: float64
max: 0.9978448
min: 0.0018775834
count > 0.5: 141
count > 0.1: 344
count > 0.01: 3157
